## 載入套件

In [13]:
# predict
import os,shutil,sys#建資料夾用
import time #sleep用
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd
from keras.models import load_model#載入h5檔案使用
import tensorflow as tf
import glob

## 初始設定載入

In [11]:
'''
首先需要一個training後的.h5結構權重檔案
參考權重檔(另一種載入方法)
https://stackoverflow.com/questions/53295570/userwarning-no-training-configuration-found-in-save-file-the-model-was-not-c
'''
infer_model = load_model('H:\\predict_model\\imt_yolo3.h5',compile=False)

In [14]:
#建立資料夾
def emtydir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
        time.sleep(2)
    os.mkdir(dirname)
#測試資料夾
def load_dataset(path):
    try:
        file=glob.glob(path+'*.jpg')
        count=0
        for i,f in enumerate(file):
            count=count+i
        print("路徑正常有",i,"個影像")
    except:
        print("檔案問題")

In [15]:
#輸入進來的測試路徑
input_path   = 'H:\\test\\'
#標框結果存檔路徑
output_path  = 'H:\\Result\\'

#新建Folder
emtydir(output_path)
load_dataset('H:\\test\\')
#輸出路徑正常有15個檔案

### Anchors設置(不需要)

In [ ]:
ROOT_PATH = 'H:/predict_model/'
# 數量
CLUSTERS = 6
# 影像尺寸
SIZE = 640

# 格式
def load_dataset(path):
    jpegimages = os.path.join(path, 'JPEGImages')
    if not os.path.exists(jpegimages):
        print('沒照片')
        sys.exit(0)

    label_file = os.listdir(labels_txt)
    print('label count: {}'.format(len(label_file)))
    dataset = []

    for label in label_file:
        with open(os.path.join(labels_txt, label), 'r') as f:
            txt_content = f.readlines()

        for line in txt_content:
            line_split = line.split(' ')
            roi_with = float(line_split[len(line_split)-2])
            roi_height = float(line_split[len(line_split)-1])
            if roi_with == 0 or roi_height == 0:
                continue
            dataset.append([roi_with, roi_height])
            # print([roi_with, roi_height])

    return np.array(dataset)

data = load_dataset(ROOT_PATH)
out = kmeans(data, k=CLUSTERS)

print(out)
print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(out[:, 0] * SIZE, out[:, 1] * SIZE))

ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))

## 比對缺值

In [ ]:
#比對兩list是否有不重複值
def different(list_1, list_2): 
    return (list(set(list_1).symmetric_difference(set(list_2))))

## 製作撈座標function

In [1]:
#傳boxes進來
def coordinate(boxes):
    print("==========驗證==========")
    labels = ['Mark_X','Mark_Y','X','Y'] 
    #box字典
    label_list=[]
    #比對
    compare=[]
    #存放結果表格，用字典建
    label_dir={'label':[],
             'x1':[],
             'y1':[],
             'x2':[],
             'y2':[],
             'score':[],
            }
    label_table=pd.DataFrame(label_dir)
    for m in range(len(boxes)):
        if boxes[m].score !=-1:
            label=boxes[m].label
            if label==0:
                label_name="Mark_X"
            elif label==1:
                label_name='Mark_Y'
            elif label==2:
                label_name="X"
            elif label==3:
                label_name="Y"
            if(boxes[m].score>0.80):
                #新增label進list
                label_list.append(label_name)
                print(label_name,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax,"Score",boxes[m].score)
            #將數值存入統一字典格式    
            statistics_dir={'label':[label_name],
                              'x1':[boxes[m].xmin],
                              'y1':[boxes[m].ymin],
                              'x2':[boxes[m].xmax],
                              'y2':[boxes[m].ymax],
                              'score':[boxes[m].score],
                                 }
            #將每一筆box資料新增到global 的結果字典
            statistics_tabel=pd.DataFrame(statistics_dir)
            label_table=label_table.append(statistics_tabel,ignore_index=True)
            

    #找list中有沒有包含所有label
    compare=different(label_list,labels)
    print("==========過濾結果===========")
    if len(compare)<1:
        #過濾重複數值(先照信心值排序，照label中重複的只留下第一筆，最後按照label排序
        label_table=label_table.sort_values('score',ascending=False).drop_duplicates('label',keep='first').sort_values(by='label')
        #重新編排index
        label_table.reset_index(drop=True,inplace=True)
        print(label_table)
        #開始量測(座標表格,標記名稱)
        point_table=cutting_measure(label_table,labels)
        return point_table
    else:
        print("有缺值",compare)
        return null

### 量測精度

In [1]:
#量測經度
def cutting_measure(table,ref_list):
        #initialize
        point_1=0
        point_2=0
        point_3=0
        point_4=0
        point_5=0
        point_6=0
        point_7=0
        point_8=0
        #放8個點位置
        point_dic={'sheet_id':[],
                   'point_1':[],
                   'point_2':[],
                   'point_3':[],
                   'point_4':[],
                   'point_5':[],
                   'point_6':[],
                   'point_7':[],
                   'point_8':[],
                   'check':[],
                  }
        #將字典做成表格
        point_table=pd.DataFrame(point_dic)

        print("===========================")
        #過濾完成開始計算距離，取過濾完的dataframe就一定剩下4個
        c_row,c_column=table.shape
        #順序(Mark_X、Mark_Y、X、Y)


        data_x1=table['x1']
        data_x2=table['x2']
        data_y1=table['y1']
        data_y2=table['y2']
        
        #算Mark_X
        Mark_X_Line=data_y1[0]+((data_y2[0]-data_y1[0])/2)
        #算Mark_Y
        Mark_Y_Line=data_x1[1]+((data_x2[1]-data_x1[1])/2)
        #算X_Line
        X_Line=data_y1[2]+((data_y2[2]-data_y1[2])/2)
        #算Y_Line
        Y_Line=data_y1[3]+((data_y2[3]-data_y1[3])/2)      
        
        #量測距離，用切割線-mark 
        x_measure=X_Line-Mark_X_Line
        y_measure=Y_Line-Mark_Y_Line
        #print("Mark_x",Mark_X_Line,"Mark_y",Mark_Y_Line,"x_line",X_Line,"y_line",Y_Line)
        #print(x_measure,'==',y_measure)
        
        #利用正負判斷角落
        if x_measure<0:
            if y_measure<0:
                corner="Upper_left_corner"
                point_2=abs(x_measure)
                point_3=abs(y_measure)
                step_dic={ 
                           'point_2':[point_2],
                           'point_3':[point_3],
                         }
            elif y_measure>0:
                corner="Upper_right_corner"
                point_1=abs(x_measure)
                point_8=abs(y_measure)
                step_dic={ 
                           'point_1':[point_1],
                           'point_8':[point_8],
                         }
        elif x_measure>0:
            if y_measure>0:
                corner="Bottom_right_corner"
                point_7=abs(x_measure)
                point_6=abs(y_measure)
                step_dic={ 
                           'point_7':[point_7],
                           'point_6':[point_6],
                         }
            elif y_measure<0:
                corner="Bottom_right_corner"
                point_5=abs(x_measure)
                point_4=abs(y_measure)
                step_dic={ 
                           'point_5':[point_5],
                           'point_4':[point_4],
                         }
        #輸出哪個角落
        print(corner)
        #把結果放到字典

        step_table=pd.DataFrame(step_dic)
        #合併dataframe
        point_table=point_table.append(step_table,ignore_index=True,sort=False)
        z1 = point_table.T
        z=z1.where(z1.notnull(),0)
        #存成csv
        df=pd.DataFrame(z).to_csv('H:/Record/point.csv',index=False)
        #回傳
        return z

## model繪製方框

In [ ]:
# Set some parameter
net_h, net_w = 608, 608 # a multiple of 32, the smaller the faster
#物件信心值、重疊度
obj_thresh,nms_thresh=0.80,0.3

anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
#[0],[1],[2]照字母順序編排標籤順序
labels = ['Mark_X','Mark_Y','X','Y'] 


# Read file
image_paths = []
if os.path.isdir(input_path): 
    for inp_file in os.listdir(input_path):
        image_paths += [input_path + inp_file]
else:
    image_paths += [input_path]

image_paths = [inp_file for inp_file in image_paths if (inp_file[-4:] in ['.jpg', '.png', '.JPG'])]

# the main loop
for image_path in image_paths:
    image = cv2.imread(image_path)
    #print(image_path)

# predict the bounding boxes
    boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]
# draw bounding boxes on the image using labels
    draw_boxes(image, boxes, labels, obj_thresh)  

# write the image with bounding boxes to file
    output_img_path = output_path +'predict_'+image_path.split('\\')[-1]
    cv2.imwrite(output_img_path, np.uint8(image))
    img=cv2.imread(output_img_path)[:,:,::-1]
    #print(output_img_path)
    
#取得BBOX座標
    #box(boxes)
    label_table=coordinate(boxes)
    print("===========================")
    print(label_table)
    #相加
#輸出
'''
==========驗證==========
Mark_X 112 180 476 205 Score 0.9712204
Y 323 -22 333 285 Score 0.960079
Mark_Y 277 80 317 289 Score 0.9996914
X -42 207 421 216 Score 0.9294017
Y 323 144 334 434 Score 0.9977327
==========過濾結果===========
    label     x1     y1     x2     y2     score
0  Mark_X  112.0  180.0  476.0  205.0  0.971220
1  Mark_Y  277.0   80.0  317.0  289.0  0.999691
2       X  -42.0  207.0  421.0  216.0  0.929402
3       Y  323.0  144.0  334.0  434.0  0.997733
===========================
Bottom_right_corner
===========================
             0
sheet_id   0.0
point_1    0.0
point_2    0.0
point_3    0.0
point_4    8.0
point_5   19.0
point_6    0.0
point_7    0.0
point_8    0.0
check      0.0

'''

## 測試輸出座標(單一)

In [1]:
#test
def box(boxes):
    for m in range(len(boxes)):
        if boxes[m].score !=-1:
            label=boxes[m].label
            if label==0:
                label_name="Mark_X"
                Mark_X_Line=boxes[m].ymax-boxes[m].ymin
                print("Mark_X",Mark_X_Line)
            elif label==1:
                label_name="Mark_Y"
                Mark_Y_Line=boxes[m].xmax-boxes[m].xmin
                print("Mark_Y",Mark_Y_Line)
            elif label==2:
                label_name="X"
                X_Line=boxes[m].ymax-boxes[m].ymin
                print("X",X_Line)
            elif label==3:
                label_name="Y"
                Y_Line=boxes[m].xmax-boxes[m].xmin
                print("Y",Y_Line)
            if(boxes[m].score>0.80):
                print(label_name,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax,"評分",boxes[m].score)
            
#輸出
'''
X    1 -3 188 509 194
Y    2 234 29 251 291
Mark 0 244 158 327 192
X    1 -25 185 329 197
Y    2 237 113 251 432
'''